In [59]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from data_cleaning import *
import implementations as imp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import data

In [2]:
y_train, x_train, ids_train = load_csv_data("../data/train.csv")
y_test, x_test, ids_test = load_csv_data("../data/test.csv")

In [88]:
y_test

array([1., 1., 1., ..., 1., 1., 1.])

# Clean test data and add features

In [60]:
NUM_JETS = 4

PRI_jet_num_train = np.array([x_train[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_train)
del_x_train = np.delete(x_train, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

replaced_x_train = replace_undefined_with_mean(del_x_train, UNDEFINED_VALUE)

norm_x_train, train_data_mean, train_data_std = mean_std_normalization(replaced_x_train)

print(norm_x_train[0][0])
print(norm_x_train.shape)

[[2 1 1 ... 1 0 0]]
(250000, 29)
0.4567790596384003
(250000, 29)


# Do the same for the test data

In [94]:
PRI_jet_num_test = np.array([x_test[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_test)
print(PRI_jet_num_test.shape)
del_x_test = np.delete(x_test, COLUMN_TO_DROP, axis=1)
print(del_x_test.shape)

replaced_x_train = replace_undefined_with_mean(del_x_test, UNDEFINED_VALUE)

norm_x_test, test_data_mean, test_data_std = mean_std_normalization(replaced_x_test, train_data_mean, train_data_mean)
print(norm_x_test[0][0])
print(norm_x_test.shape)



[[0 1 0 ... 0 1 0]]
(1, 568238)
(568238, 29)
-1.0090582089055866
(568238, 29)


# Split data based on PRI_jet_num

ValueError: could not broadcast input array from shape (99913,29) into shape (1)

In [71]:
#remove columns that contain only 0
print(norm_x_train.shape)
    

(250000, 29)


# Train model

In [91]:
init_w = np.ones(30)
max_iters = 100
gamma = 0.000001
degree = 10
b_size = 1
ratio = 0.8
seed = 1

for i in range(NUM_JETS):
    
    curr_x = norm_x_train[PRI_jet_num_train[0,:]==i]
    curr_y = y_train[PRI_jet_num_train[0,:]==i]
    
    curr_x_test = norm_x_test[PRI_jet_num_test[0,:]==i]
    
    (tr_x, tr_y, te_x,te_y) = split_data(curr_x, curr_y, ratio, seed)

    px_tr = build_poly(curr_x,degree)
    px_te = build_poly(curr_x_test,degree)
    
    w_list = []
    _, w = imp.ridge_regression(curr_y,px_tr, gamma)
    w_list.append(w)
    gamma = gamma + 0.000002

    #TODO: PICK BEST w!!
    y_test[PRI_jet_num_test[0,:]==i] = predict_labels(w, px_te)
    


# test accuracy

In [ ]:
    
    
print(w_list[0].shape)
n = len(y_pred)
correct = 0
for j in range(len(w_list)):
    correct = 0
    y_pred = predict_labels(w_list[j], px_te)
    for i in range(n):
        if y_pred[i] == te_y[i]:
            correct = correct + 1
        ratio = correct/n
    print(j, ratio)

y_pred = predict_labels(w, px_te)
print(w_list[0].shape)
n = len(y_pred)
correct = 0
ratio = []
for j in range(len(w_list)):
    correct = 0
    y_pred = predict_labels(w_list[j], px_te)
    for i in range(n):
        if y_pred[i] == te_y[i]:
            correct = correct + 1
        ratio.append(correct/n)
    #print(j, ratio)
print(max(ratio))

In [92]:
create_csv_submission(ids_test, y_test, "y_pred.csv")

In [93]:
y_test

array([-1., -1.,  1., ...,  1., -1., -1.])